In [1]:
# Human specific
import inspect

environment_name = "MeetingProblem"
specific = inspect.cleandoc("""\
                Two agents are tasked with manipulating blocks arranged in stacks on a table, from one configuration to another. 
                Each block is uniquely labelled by a letter. Both agents can only interact with blocks at the top of each stack, and only interact with one block at a time. 
                Additionally, one agent can only interact with vowel blocks, and the other can only interact with consonant blocks. 
                Initially there are blocks A, B, C, O. A is on the table. B is on top of A, and C and O are on the table. 
                The goal is to have A on the table, B on the table, C on top of O, and O on top of B.
                """)  # This prompt is ignored if the environment_name already corresponds to an existing environment.
format = "json"

In [2]:
# Generate the json representation of the environment
from pathlib import Path

from src.llm_plan.planner import Planner
from src.llm_plan.llm import ChatGPT
from src.llm_plan.config import ENVIRONMENTS_JSON_PATH

model = ChatGPT("gpt-5-mini")  # This is one of the many models you can use now
planner = Planner()

plan_path = Path(f"{environment_name}.{format}")
full_path = ENVIRONMENTS_JSON_PATH / plan_path

# Skip if the plan already exists
if not full_path.exists():
    planner.generate_representation(model, specific, environment_name, format=format)
else:
    print(f"{full_path} already exists. Skipping generation.")
    print("[Warning]: The prompt `specific` will be ignored!")

environments/static/MeetingProblem.json already exists. Skipping generation.
[Warning]: The prompt `specific` will be ignored!


In [3]:
# Generate the environment plan
from src.llm_plan.environment import Environment

env = Environment(f"./environments/static/{environment_name}.json")
print("Plan:\n", env.plan)

Plan:
 [['Alice.pddl', 'Bob.pddl', 'orchestrator.pddl']]


In [4]:
# Hypervisor
import re
import subprocess
from pathlib import Path

from src.llm_plan.hypervisor import Hypervisor
from src.llm_plan.config import UNIVERSAL_VALIDATOR_BIN, SOLVER_BINARY, SOLVER_ARGS
from src.llm_plan.parser import PDDLParser

model = ChatGPT("gpt-4o")  # switch to a less powerful model for planning
pddl_parser = PDDLParser()

BASE_FOLDER = Path(f"./tmp/{env.name}")
BASE_FOLDER.mkdir(parents=True, exist_ok=True)

# Fist plan to collect domain and problem
print("Generating the first plan.")
responses = planner.plan(model, env)
final_plan = responses["pddl_orchestrator"]
domain, problem = pddl_parser.parse(final_plan, from_file=False)

budget = 10
for _ in range(budget):
    prompt_args_hypervisor = {
        "plan": "No plan yet.",
        "specification": env.config_data,
        "pddl_domain": domain,
        "pddl_problem": problem,
        "syntax_errors": "No error file yet.",
        "pddl_logs": "No log file yet.",
        "history": [],
    }
    hypervisor = Hypervisor(prompt_args_hypervisor)
    response = hypervisor.run(model)
    
    # Dynamically instantiate the agent class
    match = re.search(r"<class>(.*?)</class>", response, re.DOTALL)
    if match:
        agent_name = match.group(1).strip()
    else:
        print(f"[Warning] No agent class found in the response ({agent_name}). Using default AgentDeepThinkPDDL.")
        agent_name = "AgentDeepThinkPDDL"
        
    print("Agent: ", agent_name)
    print("Generating the plan with the agent.")
    agent_class = hypervisor.agents[agent_name]
    
    required_args = {}
    for arg in agent_class.required_args.keys():
        agent_class.required_args[arg] = prompt_args_hypervisor[arg]
        
    new_agent = agent_class(model, agent_class.required_args)
    response = new_agent.run()
    
    # Get domain and plan
    domain, problem = pddl_parser.parse(response, from_file=False)
    prompt_args_hypervisor["pddl_domain"] = domain
    prompt_args_hypervisor["pddl_problem"] = problem
    
    # Run the pddl planner
    with open(BASE_FOLDER / "problem.pddl", "w") as f:
        f.write(problem)

    with open(BASE_FOLDER / "domain.pddl", "w") as f:
        f.write(domain)
        
    # Launch the solver
    command = [
        SOLVER_BINARY,
        *SOLVER_ARGS,
        BASE_FOLDER / "sas_plan",
        BASE_FOLDER / "domain.pddl",
        BASE_FOLDER / "problem.pddl",
    ]

    with open(BASE_FOLDER / "logs.txt", "w") as logfile:
        subprocess.run(command, stdout=logfile, stderr=subprocess.STDOUT)
    
    # Validate the plan with uVAL
    command = f"{UNIVERSAL_VALIDATOR_BIN} -cv \
    {BASE_FOLDER / 'domain.pddl'} \
    {BASE_FOLDER / 'problem.pddl'} \
    {BASE_FOLDER / 'sas_plan'}"

    out = subprocess.run(command, shell=True, capture_output=True, text=True)

    # This part won't be printed at test time
    if out.stderr:
        pddl_error = out.stderr
        print(
            f"The validation found a problem with the plan: {out.stderr}"
        )
    else:
        pddl_error = "No error found."
        print("The plan is valid.")
        print("[stdout]", out.stdout)
        
    with open(BASE_FOLDER / "logs.txt", "r") as f:
        pddl_logs = f.read()
    
    # Syntax errors and logs
    prompt_args_hypervisor["syntax_errors"] = pddl_error
    prompt_args_hypervisor["pddl_logs"] = pddl_logs
    
    # History
    hypervisor.history.append(agent_name)
    prompt_args_hypervisor["history"] = hypervisor.history
    

Generating the first plan.
Agent:  AgentEnforceMultiAgency
Generating the plan with the agent.
The validation found a problem with the plan: libc++abi: terminating due to uncaught exception of type parser::pddl::UnknownToken: ALICE does not name a known token

Agent:  AgentEnforceMultiAgency
Generating the plan with the agent.
The validation found a problem with the plan: libc++abi: terminating due to uncaught exception of type parser::pddl::UnknownToken: ALICE does not name a known token

Agent:  AgentEnforceMultiAgency
Generating the plan with the agent.
The plan is valid.
[stdout] Checking plan: tmp/MeetingInConnectedRooms/sas_plan
Plan executed successfully - checking goal
Success: Plan valid
Value: 2.000000


Agent:  AgentEnforceMultiAgency
Generating the plan with the agent.
The validation found a problem with the plan: libc++abi: terminating due to uncaught exception of type parser::pddl::UnknownToken: ALICE does not name a known token

Agent:  AgentEnforceMultiAgency
Generating

In [5]:
# Report the natural language plan
from src.llm_plan.agent import AgentNaturalLanguage

with open(BASE_FOLDER / "problem.pddl", "r") as f:
    problem = f.read()

with open(BASE_FOLDER / "domain.pddl", "r") as f:
    domain = f.read()
    
with open(BASE_FOLDER / "sas_plan", "r") as f:
    plan = f.read()
    
prompt_args = {
    "specification": env.config_data,
    "pddl_domain": domain,
    "pddl_problem": problem,
    "pddl_plan": plan,
}

hypervisor_to_nl = AgentNaturalLanguage(llm=model, prompt_args=prompt_args)

natural_plan = hypervisor_to_nl.run()

with open(BASE_FOLDER / "refined_nl_plan.txt", "w") as f:
    f.write(natural_plan)